In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')



In [26]:
tokens = nlp("cook on medium to high heat for 7-8 minutes")
print([(w.text, w.dep_) for w in tokens])

from spacy import displacy
displacy.render(tokens, style='dep')
displacy.render(tokens, style='ent')
print(tokens[4].text)
for child in tokens[4].children:
    print(child.dep_)


[('cook', 'ROOT'), ('on', 'prep'), ('medium', 'amod'), ('heat', 'pobj'), ('for', 'prep'), ('7', 'nummod'), ('-', 'punct'), ('8', 'prep'), ('minutes', 'pobj')]


for
pobj


In [27]:
tokens = nlp("diced and peeled fresh ginger root, cut into 2 inch pieces")
print([(w.text, w.dep_) for w in tokens])

from spacy import displacy
displacy.render(tokens, style='dep')
displacy.render(tokens, style='ent')

for child in tokens[4].children:
    print(child.dep_)

[('diced', 'ROOT'), ('and', 'cc'), ('peeled', 'conj'), ('fresh', 'amod'), ('ginger', 'compound'), ('root', 'dobj'), (',', 'punct'), ('cut', 'dep'), ('into', 'prep'), ('2', 'nummod'), ('inch', 'compound'), ('pieces', 'pobj')]


In [28]:
import re

def fraction_to_decimal(fraction):
    unicode_fractions = {
        '½': 0.5, '⅓': 1/3, '⅔': 2/3, '¼': 0.25, '¾': 0.75,
        '⅕': 0.2, '⅖': 0.4, '⅗': 0.6, '⅘': 0.8, '⅙': 1/6, 
        '⅚': 5/6, '⅛': 0.125, '⅜': 0.375, '⅝': 0.625, '⅞': 0.875
    }
    if fraction in unicode_fractions:
        return unicode_fractions[fraction]

    if '/' in fraction:
        numerator, denominator = fraction.split('/')
        return float(numerator) / float(denominator)

    return 0

def replace_quantity_with_decimal(ingredient):
    # Updated pattern to match whole number with optional fraction, standalone fraction, or standalone whole number
    quantity_pattern = r'((\d+\s*)(\d+/\d+|[\u00BC-\u00BE\u2150-\u215E]))|((\d+/\d+|[\u00BC-\u00BE\u2150-\u215E]))|(\d+)'

    def replace_match(match):
        # Extract the whole number and fraction parts
        whole_number = match.group(6) or match.group(2)
        fraction = match.group(3) or match.group(5) 

        decimal_whole_number = int(whole_number) if whole_number else 0
        decimal_fraction = fraction_to_decimal(fraction) if fraction else 0

        total_quantity = decimal_whole_number + decimal_fraction
        return str(total_quantity)

    # Replace the quantity with the decimal sum
    return re.sub(quantity_pattern, replace_match, ingredient, 1)

# Example usage
ingredients = ["1 ½ cups coarsely chopped onion", "½ cup chopped carrot", "2 cups flour", "3/4 cup sugar", "1½ cup (102 g) panko"]
for ingredient in ingredients:
    print(replace_quantity_with_decimal(ingredient))

1.5 cups coarsely chopped onion
0.5 cup chopped carrot
2 cups flour
0.75 cup sugar
1.5 cup (102 g) panko


In [29]:
import re

def fraction_to_decimal(fraction):
    # Handle Unicode fractions
    unicode_fractions = {
        '½': 0.5, '⅓': 1/3, '⅔': 2/3, '¼': 0.25, '¾': 0.75,
        '⅕': 0.2, '⅖': 0.4, '⅗': 0.6, '⅘': 0.8, '⅙': 1/6, 
        '⅚': 5/6, '⅛': 0.125, '⅜': 0.375, '⅝': 0.625, '⅞': 0.875
    }
    if fraction in unicode_fractions:
        return unicode_fractions[fraction]

    # Handle ASCII fractions
    if '/' in fraction:
        numerator, denominator = fraction.split('/')
        return float(numerator) / float(denominator)

    return 0

def replace_fraction_with_decimal(ingredient):
    # Regular expression to identify fractions
    fraction_pattern = r'(\d+/\d+|[\u00BC-\u00BE\u2150-\u215E])'

    # Function to use with re.sub
    def replace_match(match):
        fraction = match.group(0)
        decimal = fraction_to_decimal(fraction)
        return str(decimal)

    # Replace all fractions in the string with their decimal equivalents
    return re.sub(fraction_pattern, replace_match, ingredient)

ingredient = "1 ½ cups coarsely chopped onion"
ingredient_with_decimal = replace_fraction_with_decimal(ingredient)
print(ingredient_with_decimal)

1 0.5 cups coarsely chopped onion


In [30]:
from utils.ingredient import RecipeIngredient

ing = RecipeIngredient.from_string("2 tablespoons finely chopped Italian parsley")

ModuleNotFoundError: No module named 'ontologies'

In [ ]:
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}]
matcher.add("HelloWorld", [pattern])

doc = nlp("Hello, world! Hello world!")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

print(len(matches))

15578876784678163569 HelloWorld 0 3 Hello, world
1
